In [1]:
import requests as req
import json
import pandas as pd

In [2]:
r = req.get('https://footballapi.pulselive.com/football/stats/match/93526',
            headers={'origin':'https://www.premierleague.com'})

In [3]:
conteudo = json.loads(r.content)

In [4]:
conteudo['entity']

{'gameweek': {'id': 12289,
  'compSeason': {'label': '2023/24',
   'competition': {'abbreviation': 'EN_PR',
    'description': 'Premier League',
    'level': 'SEN',
    'source': '',
    'id': 1},
   'id': 578},
  'gameweek': 21},
 'kickoff': {'completeness': 3,
  'millis': 1705149000000,
  'label': 'Sat 13 Jan 2024, 12:30 GMT'},
 'provisionalKickoff': {'completeness': 3,
  'millis': 1705149000000,
  'label': 'Sat 13 Jan 2024, 12:30 GMT'},
 'teams': [{'team': {'name': 'Chelsea',
    'club': {'name': 'Chelsea',
     'shortName': 'Chelsea',
     'abbr': 'CHE',
     'id': 4},
    'teamType': 'FIRST',
    'shortName': 'Chelsea',
    'id': 4},
   'score': 1},
  {'team': {'name': 'Fulham',
    'club': {'name': 'Fulham', 'shortName': 'Fulham', 'abbr': 'FUL', 'id': 34},
    'teamType': 'FIRST',
    'shortName': 'Fulham',
    'id': 34},
   'score': 0}],
 'replay': False,
 'ground': {'name': 'Stamford Bridge',
  'city': 'London',
  'source': 'OPTA',
  'id': 30},
 'neutralGround': False,
 'status

In [8]:
# faz request do json
def request_pl_match(url):
  r = req.get(url,headers={'origin':'https://www.premierleague.com'})
  conteudo = json.loads(r.content)
  return conteudo

#limpa json de info
def clean_match_info(response):
  gw = [response['entity']['gameweek']['gameweek']]
  id = [response['entity']['id']]
  info_teams = [[i['team']['id'],i['team']['shortName'],i['score']] for i in response['entity']['teams']]
  info_kickoff = [response['entity']['kickoff']['label']]
  clean_info = pd.DataFrame([id+gw+i+info_kickoff for i in info_teams],
                            columns = ['match_id','gw_id','team_id','team_name','score','timestamp'])

  return clean_info

#limpa json de dados
def clean_match_stats(response):
  stats = [response['data'].get(i)['M'] for i in response['data']]
  team_id = [i for i in response['data']]

  stats_team_a = {i['name']:i['value'] for i in stats[0]}
  stats_team_b = {i['name']:i['value'] for i in stats[1]}

  stats_team_a['team_id'] = int(team_id[0])
  stats_team_b['team_id'] = int(team_id[1])

  stats_final = pd.DataFrame([stats_team_a,stats_team_b])

  return stats_final

#junta tudo
def scrape_match(match_to_scrape):
  conteudo = request_pl_match(match_to_scrape)

  match_info = clean_match_info(conteudo)
  match_stats = clean_match_stats(conteudo)

  full_match_data = pd.merge(match_info,match_stats,on='team_id')

  return full_match_data

In [6]:
df_pl_data_gw21 = pd.DataFrame()

ids = list(range(93321, 93520))

for id in ids:
  url = 'https://footballapi.pulselive.com/football/stats/match/' + str(id)
  try:
    df_pl_data_gw21 = pd.concat([df_pl_data_gw21,scrape_match(url)])
  except:
    continue


In [7]:
df_pl_data_gw21

,match_id,gw_id,team_id,team_name,score,timestamp,formation_used,poss_lost_all,duel_won,poss_lost_ctrl,...,att_ibox_own_goal,foul_throw_in,att_obp_goal,att_cmiss_high_left,att_pen_post,att_freekick_goal,att_freekick_post,att_obx_left,att_lg_right,att_obx_right
0,93321,1,43,Burnley,0,"Fri 11 Aug 2023, 20:00 BST",541.0,112.0,36.0,112.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93321,1,11,Man City,3,"Fri 11 Aug 2023, 20:00 BST",4231.0,107.0,44.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,93322,1,1,Arsenal,2,"Sat 12 Aug 2023, 13:00 BST",433.0,119.0,47.0,119.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93322,1,15,Nott'm Forest,1,"Sat 12 Aug 2023, 13:00 BST",343.0,106.0,56.0,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,93323,1,127,Bournemouth,1,"Sat 12 Aug 2023, 15:00 BST",4231.0,168.0,55.0,168.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,93517,20,12,Man Utd,1,"Sat 30 Dec 2023, 17:30 GMT",4231.0,147.0,42.0,147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,93518,20,21,Tottenham Hotspur,3,"Sun 31 Dec 2023, 14:00 GMT",433.0,138.0,58.0,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93518,20,127,Bournemouth,1,"Sun 31 Dec 2023, 14:00 GMT",4231.0,135.0,57.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,93519,20,25,West Ham,0,"Tue 2 Jan 2024, 19:30 GMT",4231.0,113.0,37.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# salvando arquivo em csv
df_pl_data_gw21.to_csv('pl_data.csv',index=False)

NameError: name 'df_pl_data_gw21' is not defined

In [ ]:
# salvando arquivo em txt com o nome das colunas obtidas
columns = list(df_pl_data_gw21.columns)

with open('cols.txt',mode='w',encoding='utf-8') as cols:
  for i in columns:
    cols.write(f'{i}\n')